# Goals
+ drop na rows
+ lowercase language
+ file per language

In [8]:
import os, yaml
import pandas as pd
conffile = '../config/onboarder_thanker_201906_hardlaunch.yaml'
qual_map_f = '../config/qualtrics_to_interal_field_map.yaml'

In [9]:
conf = yaml.safe_load(open(conffile,'r'))
qualtrics_map = yaml.safe_load(open(qual_map_f,'r'))

In [10]:
pdir = conf['dirs']['project']
idir = conf['dirs']['input']

In [11]:
langs = conf['langs']

In [19]:
def clean_raw_df(rdf):
    print(f'raw file has shape {rdf.shape} before na drop')
    rdf.dropna(inplace=True)
    print(f'raw file had shape {rdf.shape} after na drop')

    rdf['lang'] = rdf['lang'].apply(lambda s: s.lower())
    rdf['user_email'] = rdf['user_email'].apply(lambda s: s.lower())
    
    print(f'raw file has shape {rdf.shape} before drop duplicates')
    rdf.drop_duplicates(subset=['user_email'], keep='first', inplace=True)
    rdf.drop_duplicates(subset=['user_name'], keep='first', inplace=True)
    print(f'raw file had shape {rdf.shape} after drop duplicates')
    return rdf

def clean_survey(sdf):
    print(f'raw file has shape {sdf.shape} before na drop')
    sdf.dropna(inplace=True)
    print(f'raw file had shape {sdf.shape} after na drop')
    columns_to_norm = ['activities_1', 'activities_2', 'activities_3', 'activities_4', 'activities_5', 'trust_1', 'trust_2', 'trust_3', 'trust_4', 'burnout_1', 'burnout_2']
    for col in columns_to_norm:
        sdf[col] = sdf[col].apply(lambda x: x if isinstance(x, int) else int(x) if isinstance(x, float) else int(x[0]))
    return sdf

In [20]:
for lang in langs:
    print(f'workin on {lang}')
    lang_raw = os.path.join(pdir, idir, conf['langs'][lang]['consented_file_raw'])
    survey_raw = os.path.join(pdir, idir, conf['langs'][lang]['survey_file_raw'])
    
    rdf = pd.read_csv(lang_raw, header=0, skiprows=lambda r: r in(1,2))
    rdf_clean = clean_raw_df(rdf)
    rdf_clean_f = os.path.join(pdir, idir, conf['langs'][lang]['consented_file'])
    rdf_clean.to_csv(rdf_clean_f, index=False)
    
    sdf = pd.read_csv(survey_raw, header=0, skiprows=lambda r: r in(1,2))
    sdf_clean = clean_survey(sdf)
    sdf_clean_f = os.path.join(pdir, idir, conf['langs'][lang]['survey_file'])
    sdf_clean.to_csv(sdf_clean_f, index=False)

workin on pl
raw file has shape (79, 14) before na drop
raw file had shape (79, 14) after na drop
raw file has shape (79, 14) before drop duplicates
raw file had shape (74, 14) after drop duplicates
raw file has shape (72, 24) before na drop
raw file had shape (71, 24) after na drop
workin on ar
raw file has shape (42, 14) before na drop
raw file had shape (35, 14) after na drop
raw file has shape (35, 14) before drop duplicates
raw file had shape (32, 14) after drop duplicates
raw file has shape (36, 24) before na drop
raw file had shape (30, 24) after na drop
workin on de
raw file has shape (317, 14) before na drop
raw file had shape (317, 14) after na drop
raw file has shape (317, 14) before drop duplicates
raw file had shape (308, 14) after drop duplicates
raw file has shape (304, 24) before na drop
raw file had shape (304, 24) after na drop
workin on fa
raw file has shape (65, 15) before na drop
raw file had shape (62, 15) after na drop
raw file has shape (62, 15) before drop dupl

In [17]:
%debug

> <ipython-input-15-523aa1eae158>(21)<lambda>()
     18     print(f'raw file had shape {sdf.shape} after na drop')
     19     columns_to_norm = ['activities_1', 'activities_2', 'activities_3', 'activities_4', 'activities_5', 'trust_1', 'trust_2', 'trust_3', 'trust_4', 'burnout_1', 'burnout_2']
     20     for col in columns_to_norm:
---> 21         sdf[col] = sdf[col].apply(lambda x: x if isinstance(x, int) else int(x[0]))
     22     return sdf

ipdb> col
*** NameError: name 'col' is not defined
ipdb> u
> /home/paprika/workspace/civilservant-wikipedia-edit-sync/ipython/pandas/_libs/lib.pyx(2217)pandas._libs.lib.map_infer()

ipdb> ls
*** NameError: name 'ls' is not defined
ipdb> l

ipdb> up
> /home/paprika/.local/share/virtualenvs/civilservant-wikipedia-edit-sync-FxtF2vf9/lib/python3.7/site-packages/pandas/core/series.py(3591)apply()
   3589             else:
   3590                 values = self.astype(object).values
-> 3591                 mapped = lib.map_infer(values, f, convert=c

# End of softlaunch code

In [14]:
# consent = pd.read_csv(os.path.join(pdir, idir, 'thanker_consent.csv'))
# survey = pd.read_csv(os.path.join(pdir, idir, 'thanker_pre_survey.csv'))

FileNotFoundError: [Errno 2] File b'/home/paprika/Tresors/CivilServant/projects/wikipedia-integration/gratitude-study/Data Drills/thanker_softlaunch/julia_input/thanker_consent.csv' does not exist: b'/home/paprika/Tresors/CivilServant/projects/wikipedia-integration/gratitude-study/Data Drills/thanker_softlaunch/julia_input/thanker_consent.csv'

(535, 13)
(532, 13)


In [48]:
print(consent.shape)
survey.dropna(inplace=True)
print(consent.shape)

(532, 13)
(532, 13)


In [51]:
consent['lang'] = consent['lang'].apply(lambda s: s.lower())
survey['lang'] = survey['lang'].apply(lambda s: s.lower())

In [54]:
consent['lang'].value_counts()
survey['lang'].value_counts()

de    302
pl     78
fa     77
ar     75
Name: lang, dtype: int64

In [60]:
langs = consent['lang'].value_counts().index.tolist()

In [68]:
for lang in langs:
    sub_consent = consent[consent['lang']==lang]
    sub_survey = survey[survey['lang']==lang]
    fname_consent = f'{lang}_consented_thankers.csv'
    fname_survey = f'{lang}_pre_surveys.csv'
    sub_consent.to_csv(os.path.join(pdir, idir, fname_consent))
    sub_survey.to_csv(os.path.join(pdir, idir, fname_survey))